In [ ]:
#plot training data predictions vs true
n_features = trainX_all.shape[2]
predicted_RULs = []
true_RULs = []
for i in range(6):
    unit_data = trainX_all[i].reshape(1, n_past, n_features)  # Reshape to 3D array
    predicted_RUL = model.predict(unit_data, verbose = 0, batch_size = 32)
    predicted_RULs.append(predicted_RUL.flatten()[0])  # Flatten the prediction
    true_RULs.append(trainY_all[i])

# Plotting
plt.figure(figsize=(10, 5))
plt.plot(predicted_RULs, label='Predicted RUL')
plt.plot(true_RULs, label='True RUL', linestyle='--')
plt.title(f'Predicted vs True RUL for Unit 1')
plt.xlabel('Section Number')
plt.ylabel('RUL')
plt.legend()
plt.show()

In [ ]:
predicted_RULs = []
true_RULs = []
for x in range(7):
  x += 1
  for i in range(300):
    if trainX_all[i][0][0] == x:
      unit_data = trainX_all[i].reshape(1, n_past, n_features)  # Reshape to 3D array
      predicted_RUL = model.predict(unit_data, verbose = 0, batch_size = 32)
      predicted_RULs.append(predicted_RUL.flatten()[0])  # Flatten the prediction
      true_RULs.append(trainY_all[i])
    
  plt.figure(figsize=(5, 5))
  plt.plot(predicted_RULs, label='Predicted RUL')
  plt.plot(true_RULs, label='True RUL', linestyle='--')
  plt.title(f'Predicted vs True RUL')
  plt.xlabel('Cycle')
  plt.ylabel('RUL')
  plt.legend()
  plt.show()
  predicted_RULs = []
  true_RULs = []

In [ ]:
test=np.array( true_RULs)
splits=[]
test2= np.array(predicted_RULs)
for i in range(1,len(test)):
  if test[i] > test[i-1]:
    splits.append(i)

test_split = np.split(test, splits)
test2_split = np.split(test2, splits)


for i in range(len(test2_split)):
    plt.figure(figsize=(10, 10))
    plt.plot(test2_split[i], label='Predicted RUL')
    plt.plot(test_split[i], label='True RUL')
    plt.title(f'Predicted RUL vs True RUL for Unit {i+1}')
    plt.xlabel('Cycle Time')
    plt.ylabel('Remaining Useful Life (RUL)')
    plt.legend()
    plt.show()

In [ ]:
#test data set without labels
data_test = pd.read_csv('test_FD001.txt', delimiter=' ', header=None)

name_list = ['unit', 'time', 'setting0', 'setting1', 'setting2']
for i in range(number_of_sensors):
  name_list.append('sensor'+str(i))
name_list.append('empty1')
name_list.append('empty2')
data_test.columns = name_list

data_test = data_test.drop(['empty1'], axis=1)
data_test = data_test.drop(['empty2'], axis=1)

#choose which columns of data you want to keep and which you want to drop.
#Training with everything works but might not be the best choice.
data_test = data_test.drop(['setting2', 'sensor0','sensor4', 'sensor9', 'sensor15', 'sensor17', 'sensor18'], axis=1)

#normalization of the training data on selected sensors and settings
columns_to_scale = data_test.columns.drop(['unit', 'time'])
scaler = StandardScaler().fit(data_test[columns_to_scale])
data_test_scaled = data_test.copy()
data_test_scaled[columns_to_scale] = scaler.transform(data_test[columns_to_scale])

columns = data_test_scaled.shape[1]

##########################################
data_test_list = []
time_steps_data = np.empty((2, columns))

for i in range(1,len(data_test_scaled)):

  if i == len(data_test_scaled)-1:
    data_test_list.append(time_steps_data)

  elif data_test_scaled["unit"][i] != data_test_scaled['unit'][i-1]:
    time_steps_data = time_steps_data[1:]
    data_test_list.append(time_steps_data)
    time_steps_data = np.empty(columns)

  time_steps_data = np.vstack((time_steps_data, data_test_scaled.iloc[i]))


print('data_test_list length == {}.'.format(len(data_test_list)))
print('data_test_list first element shape == {}.'.format(data_test_list[0].shape))

data_test_list length == 100.
data_test_list first element shape == (31, 19).
